### ALL IMPORTS(imo its more convenient to have all imports at the top)

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
import pandas as pd

from oanda_candles import Pair, Gran, CandleClient
import config

from oandapyV20 import API
import oandapyV20.endpoints.accounts as accounts

from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
import pandas as pd
from datetime import datetime, timedelta


### FUNCTION FOR DETECTING SWEEPS

In [34]:
def detect_sweeps_improved(df, length=5):
    """
    Enhanced sweep detector with better signal quality
    """
    signals = []
    pivot_highs = []
    pivot_lows = []
    
    for i in range(length + 1):
        signals.append(0)
    
    def is_pivot_high(window, mid_idx):
        mid_high = window.iloc[mid_idx]['High']
        left_section = window.iloc[:mid_idx]
        right_section = window.iloc[mid_idx+1:]
        return (mid_high > left_section['High'].max() and 
                mid_high > right_section['High'].max())
    
    def is_pivot_low(window, mid_idx):
        mid_low = window.iloc[mid_idx]['Low']
        left_section = window.iloc[:mid_idx]
        right_section = window.iloc[mid_idx+1:]
        return (mid_low < left_section['Low'].min() and 
                mid_low < right_section['Low'].min())
    
    # Calculate average volatility
    df['TR'] = np.maximum(
        df['High'] - df['Low'],
        np.maximum(
            abs(df['High'] - df['Close'].shift(1)),
            abs(df['Low'] - df['Close'].shift(1))
        )
    )
    df['ATR'] = df['TR'].rolling(14).mean()
    
    # Main detection loop
    for i in range(length + 1, len(df)):
        current_candle = df.iloc[i]
        window = df.iloc[i-length-1:i]
        mid_idx = len(window) // 2
        
        signal = 0
        
        # Update pivot points
        if is_pivot_high(window, mid_idx):
            pivot_price = window.iloc[mid_idx]['High']
            pivot_highs.append({
                'price': pivot_price,
                'index': i-length//2,
                'active': True
            })
            
        if is_pivot_low(window, mid_idx):
            pivot_price = window.iloc[mid_idx]['Low']
            pivot_lows.append({
                'price': pivot_price,
                'index': i-length//2,
                'active': True
            })
        
        # Enhanced sweep detection
        vol_threshold = df['Volume'].rolling(20).mean().iloc[i] * 1.2
        
        # Bearish sweeps
        for ph in pivot_highs:
            if ph['active']:
                sweep_size = current_candle['High'] - ph['price']
                if (sweep_size > 0 and
                    sweep_size < current_candle['ATR'] * 0.5 and
                    current_candle['Close'] < ph['price'] and
                    current_candle['Volume'] > vol_threshold):
                    signal = 1
                    ph['active'] = False
                elif current_candle['Close'] > ph['price']:
                    ph['active'] = False
                    
        # Bullish sweeps
        for pl in pivot_lows:
            if pl['active']:
                sweep_size = pl['price'] - current_candle['Low']
                if (sweep_size > 0 and
                    sweep_size < current_candle['ATR'] * 0.5 and
                    current_candle['Close'] > pl['price'] and
                    current_candle['Volume'] > vol_threshold):
                    signal = 2
                    pl['active'] = False
                elif current_candle['Close'] < pl['price']:
                    pl['active'] = False
        
        signals.append(signal)
        
        # Clean up old pivot points
        pivot_highs = [ph for ph in pivot_highs if i - ph['index'] <= 100]
        pivot_lows = [pl for pl in pivot_lows if i - pl['index'] <= 100]
    
    return signals

### OANDA API SETUP

In [24]:
# OANDA API setup
access_token = config.API_TOKEN
accountID = config.ACCOUNT_ID
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o)) > 1)


True
True
True


### PRINT OANDA ACC SUMMARY

In [25]:
client = API(access_token)

def get_account_summary():
    # Request account details
    request = accounts.AccountDetails(accountID)
    response = client.request(request)

    # Extract relevant details
    nav = float(response['account']['NAV'])  # Net Asset Value
    unrealized_pl = float(response['account']['unrealizedPL'])  # Profit/Loss
    margin_used = float(response['account']['marginUsed'])  # Margin Used
    margin_available = float(response['account']['marginAvailable'])  # Margin Available
    account_name = response['account']['alias']  # Account Name/Alias

    # Print account details
    print(f"\nacc: '{account_name}'")
    print("-" * 40)
    print(f"folio deets:")
    print(f"- Portfolio: ${nav:,.2f}")
    print(f"- P/L: ${unrealized_pl:,.2f}")
    print(f"- Margin Used: ${margin_used:,.2f}")
    print(f"- Margin Available: ${margin_available:,.2f}")

# Call the function to print account details
get_account_summary()


acc: 'winter-break-2024-algo'
----------------------------------------
folio deets:
- Portfolio: $99,995.16
- P/L: $-4.67
- Margin Used: $145.26
- Margin Available: $99,851.92


### 1 MINUTE CANDLES: OANDA

In [28]:
def get_candles(count):
    """Get M1 (one minute) candles from Oanda"""
    params = {
        "count": count,
        "granularity": "M1"  # One minute timeframe
    }
    r = instruments.InstrumentsCandles(instrument="EUR_USD", params=params)
    client.request(r)
    return r.response['candles']

### CHECKS AND CLOSES OLD TRADES(callable function)

In [29]:
def check_and_close_old_trades(client, accountID):
    """Check for trades older than 70 minutes and close them"""
    r = trades.OpenTrades(accountID)
    open_trades = client.request(r)
    
    current_time = datetime.utcnow()
    
    for trade in open_trades['trades']:
        open_time = datetime.strptime(trade['openTime'][:19], '%Y-%m-%dT%H:%M:%S')
        if current_time - open_time > timedelta(minutes=70):
            # Close the trade
            r = trades.TradeClose(accountID, trade['id'])
            client.request(r)
            print(f"Closed trade {trade['id']} due to time limit (70 candles)")

### DETECT SIGNALS FROM 'detect_sweeps_improved' FUNCTION

In [31]:
def detect_trade_signal(candles_df):
    """
    Check if a trade can be placed based on liquidity sweep logic
    """
    df = candles_df.copy()
    df['EMA20'] = df['Close'].ewm(span=20).mean()
    signals = detect_sweeps_improved(df, length=5)
    
    current_signal = signals[-1]
    current_bar = df.iloc[-1]
    
    if current_signal == 2:  # Bullish sweep
        if abs(current_bar['Close'] - current_bar['EMA20']) / current_bar['Close'] < 0.0005:
            return True, current_bar
    
    return False, None

### CALCULATES ORDER POSITION SIZE

In [32]:
def calculate_position_size(balance, risk_percent, entry_price, stop_loss):
    """Calculate position size based on account balance and risk"""
    risk_amount = balance * (risk_percent / 100)  # Convert percent to decimal
    pip_risk = abs(entry_price - stop_loss)
    position_size = risk_amount / pip_risk
    return position_size

### FUNCTION TO ACTUALLY PLACE TRADES AUTONOMOUSLY

In [35]:
def trading_job():
    # First check and close any trades that have exceeded 70 candles
    check_and_close_old_trades(client, accountID)
    
    # Get current account balance
    request = accounts.AccountDetails(accountID)
    response = client.request(request)
    current_balance = float(response['account']['balance'])
    
    # Get M1 candles
    candles = get_candles(30)  # Get enough candles for signal detection
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low','Volume','Time'])
    
    for i, candle in enumerate(candles):
        dfstream.loc[i, 'Open'] = float(candle['mid']['o'])
        dfstream.loc[i, 'Close'] = float(candle['mid']['c'])
        dfstream.loc[i, 'High'] = float(candle['mid']['h'])
        dfstream.loc[i, 'Low'] = float(candle['mid']['l'])
        dfstream.loc[i, 'Volume'] = float(candle['volume'])
        dfstream.loc[i, 'Time'] = pd.to_datetime(candle['time'])
    
    dfstream = dfstream.astype({'Open': float, 'Close': float, 'High': float, 'Low': float, 'Volume': float})
    
    # Check for trade signal
    can_trade, current_bar = detect_trade_signal(dfstream)
    
    if can_trade:
        # Calculate trade parameters using your risk parameters
        entry_price = current_bar['Close']
        stop_loss = current_bar['Low'] - (current_bar['High'] - current_bar['Low']) * 0.1
        pip_risk = abs(entry_price - stop_loss)
        
        # Calculate position size based on 1.7% risk
        position_size = calculate_position_size(
            balance=current_balance,
            risk_percent=1.7,
            entry_price=entry_price,
            stop_loss=stop_loss
        )
        
        # Calculate take profit based on 4.5:1.7 reward:risk ratio
        take_profit = entry_price + (pip_risk * (4.5/1.7))
        
        # Execute order
        mo = MarketOrderRequest(
            instrument="EUR_USD",
            units=int(position_size),
            takeProfitOnFill=TakeProfitDetails(price=take_profit).data,
            stopLossOnFill=StopLossDetails(price=stop_loss).data
        )
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        
        print("\nTrade Executed:")
        print(f"Entry Time: {datetime.utcnow()}")
        print(f"Entry Price: {entry_price:.5f}")
        print(f"Stop Loss: {stop_loss:.5f}")
        print(f"Take Profit: {take_profit:.5f}")
        print(f"Position Size: {int(position_size)} units")
        print(f"Risk Amount: ${current_balance * 0.017:.2f}")  # 1.7% risk
        print("\nAPI Response:")
        print(rv)
    else:
        print("No valid trade setup detected")

### CALL DAT FUNCTION BOIIIIII

In [ ]:
import time

# Execute single trade
while True:
    trading_job()
    time.sleep(60)

# Uncomment below to run on schedule
# scheduler = BlockingScheduler()
# scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
# scheduler.start()

No valid trade setup detected
No valid trade setup detected
